In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [30]:
url = "https://raw.githubusercontent.com/martin2urban/pythonProject1/master/phiSlim30cols.csv"
columns= ['PhiAcc', 'ProtId', 'PathSpecies', 'PathSpeciesTaxID', 'Phenotype']
my_types = {
    "PhiAcc": str,
#   "ProtId": str,
#   "PathSpecies": str,
#  "PathSpeciesTaxID": int,

}
phi2 = pd.read_csv(
    "phiSlim30cols.csv",
    usecols=columns,
#    dtype=my_types,
    header=0,
    skiprows=[1]
)
print(phi2.columns)
phi2.describe(include='all')
phi2

Index(['PhiAcc', 'ProtId', 'PathSpeciesTaxID', 'PathSpecies', 'Phenotype'], dtype='object')


,PhiAcc,ProtId,PathSpeciesTaxID,PathSpecies,Phenotype
0,PHI:3,P26215,5017,Bipolaris zeicola,NaN
1,PHI:7,P22287,5499,Fulvia fulva,NaN
2,PHI:12,Q01886,5017,Bipolaris zeicola,NaN
3,PHI:14,P0C017,5207,Cryptococcus neoformans,NaN
4,PHI:14,P0C017,5207,Cryptococcus neoformans,NaN
...,...,...,...,...,...
20945,PHI:123548,G4MSA2,318829,Magnaporthe oryzae,NaN
20946,PHI:123548,G4MSA2,318829,Magnaporthe oryzae,NaN
20947,PHI:123549,G4MU94,318829,Magnaporthe oryzae,NaN
20948,PHI:123550,G4NB68,318829,Magnaporthe oryzae,NaN


In [ ]:
#How do I convert the columns 'PhiAcc', 'ProtId', 'PathSpecies', 'PathSpeciesTaxID' to strings?
#
columns = ['PhiAcc', 'ProtId', 'PathSpecies', 'PathSpeciesTaxID']
non_string_values_per_column = {}
for column in phi2.columns:
    non_string_values = set(phi2[column].astype(str).str.findall(r'[^a-zA-Z\s\d:]').sum())
    non_string_values_per_column[column] = non_string_values

for column, non_string_values in non_string_values_per_column.items():
    print(f"Non-string values in column '{column}':")
    for value in non_string_values:
        print(value)
    print()



In [ ]:
url = "https://raw.githubusercontent.com/martin2urban/pythonProject1/master/phiSlim30cols.csv"
columns= ['PhiAcc', 'ProtId', 'PathSpecies', 'PathSpeciesTaxID']
my_types = {
    "PhiAcc": str,
#   "ProtId": str,
#   "PathSpecies": str,
#  "PathSpeciesTaxID": int,

}
phi2 = pd.read_csv(
    "phiSlim30cols.csv",
    usecols=columns,
#    dtype=my_types,
    header=0,
)

In [ ]:
phiSlim = pd.read_csv('phiSlim30cols.csv', header=0)
phiSlim.describe()

In [ ]:
# tranform all string data in phiSlim to lowercase
phiSlim = phiSlim.applymap(lambda x: x.lower() if isinstance(x, str) else x)
phiSlim


In [ ]:
# make stats-counts report
# tabulate tally: specis x genes x interactions
# Group by 'PathSpecies' and calculate distinct 'PhiAcc' count and count of rows
data = phiSlim.groupby('PathSpecies').agg({'PhiAcc': 'nunique', 'PathSpecies': 'size'}).rename(columns={'PhiAcc': 'genes', 'PathSpecies': 'PHI.base.accession'}).reset_index()

# Sort the DataFrame by 'PHI.base.accession' in descending order
data = data.sort_values('PHI.base.accession', ascending=False)

# Rename 'PHI.base.accession' column to 'interactions'
data = data.rename(columns={'PHI.base.accession': 'interactions'})

# Write the DataFrame to a CSV file
data.to_csv(pd.Timestamp.today().strftime('%Y-%m-%d') +"phiSlim_speciesBYpathogen_geneBYinteraction_tally.csv", index=False, na_rep="")


In [ ]:
data.info()

In [ ]:
data2


In [ ]:
# tabulate tally: genus x genes x interactions
# Split data "species name
data2 = data['PathSpecies'].str.split(' ', expand=True).rename(columns={0: 'genus', 1: 'species'})
data2 = data2.iloc[:, [0, 1]]
# Group by genus and count distinct species
data3 = data2.groupby('genus').agg(Species=('species', 'nunique')).reset_index()
# Sort by descending species count
data3 = data3.sort_values('Species', ascending=False)
# Write to CSV
data3.to_csv(pd.Timestamp.today().strftime('%Y-%m-%d') +'_phiSlim_genus_no_Species.csv', index=False)

In [ ]:
data3


In [ ]:
columns = ['PhiAcc', 'PathSpecies', 'ProtId', 'HostSpeciesTaxId', 'Literature_Id', 'DiseaseName']
aggregations = {'PhiAcc': 'count', 'PathSpecies': 'nunique', 'ProtId': 'nunique', 'HostSpeciesTaxId': 'nunique', 'Literature_Id': 'nunique', 'DiseaseName': 'nunique'}

global_data = phiSlim[columns].agg(aggregations).reset_index()



In [ ]:
global_data.describe()

In [ ]:
phiSlim['DiseaseName'].nunique()
phiSlim['Phenotype'].nunique()
phiSlim['Phenotype'].astype('category').describe()
phiSlim['Phenotype'].info()


In [ ]:
url = "https://raw.githubusercontent.com/martin2urban/pythonProject1/master/phiSlim30cols.csv"
columns= ['PhiAcc', 'ProtId', 'PathSpecies', 'PathSpeciesTaxID', 'Phenotype']
my_types = {
    "PhiAcc": str,
    'Phenotype': str
#   "ProtId": str,
#   "PathSpecies": str,
#  "PathSpeciesTaxID": int,

}
phi2 = pd.read_csv(
    "phiSlim30cols.csv",
    usecols=columns,
    dtype=my_types,
    header=0,
    skiprows=0,
)
phi2['Phenotype'] = phi2['Phenotype'].astype('category')

In [ ]:
phi2.info()
phi2['PhiAcc'].dtype

In [ ]:
phi3 = phi2.loc[phi2['PhiAcc'].apply(lambda x: isinstance(x, str))]
phi3['PhiAcc'] = phi2['PhiAcc'].astype('str')
phi3
